In [2]:
import numpy as np
from synthetic_env.Env import FiniteStateFiniteActionMDP
import pickle
import matplotlib.pyplot as plt
import os
import argparse
from gridword_env.gridworld import GridWorld
import concurrent.futures
from copy import deepcopy
import random

# General Code

In [2]:
#Params
np.random.seed(25)
env = 'synthtetic' #choose between "synthetic", "gridword" and "garnett"
N = 5 #number of agengts
ep = 0.9 #Hetreogeinty transition kernels
er = 0.0 #Hetreogeinty rewards
S, A = 3, 3 # state space
gamma = 0.95 #discount factor 
init_dist = np.full(S, 1/S) #init_distribution
number_rounds = 5000
Local_steps = 10
step_size  = 0.5

In [3]:
#Generate environnements
env = FiniteStateFiniteActionMDP(S=S, A=A)
common_P = env.get_P()
common_r = env.get_r()
envs = [FiniteStateFiniteActionMDP( S=S, A=A, epsilon_p=ep, epsilon_r = er, common_transition=common_P, common_reward=common_r) for _ in range(N)]

In [4]:
#Useful functions

def compute_mrp_transition(agent, policy):
    transition_kernel =  envs[agent].get_P()
    mrp_transition = np.sum(policy[:, :, np.newaxis] * transition_kernel, axis=1)
    return mrp_transition

def compute_mrp_reward(agent, policy):
    reward = envs[agent].get_r()
    # Element-wise multiplication and sum along the actions axis
    mrp_reward = np.sum(policy * reward, axis=1)
    return mrp_reward

def compute_stationnary_distribution(agent, policy):
    mrp_transition = compute_mrp_transition(agent, policy)
    stationnary_distribution = (1- gamma) * init_dist.T @ np.linalg.inv(np.eye(S) -gamma *mrp_transition)
    return stationnary_distribution

def compute_value_function(agent, policy):
    mrp_transition = compute_mrp_transition(agent, policy)
    mrp_reward = compute_mrp_reward(agent, policy)
    return np.linalg.inv(np.eye(S) -gamma *mrp_transition) @ mrp_reward

def compute_qfunction(agent, policy):
    reward = envs[agent].get_r()
    transitions = envs[agent].get_P()
    value_function = compute_value_function(agent, policy)
    expected_future_rewards = np.sum(transitions * value_function[np.newaxis, np.newaxis, :], axis=2)
    Q_function = reward + gamma * expected_future_rewards
    return Q_function

def compute_policy(logits):
    logits_max = np.max(logits, axis=1, keepdims=True)
    exp_logits = np.exp(logits - logits_max)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

def compute_gradient(agent, logits):
    policy = compute_policy(logits)
    stationary_distribution = compute_stationnary_distribution(agent, policy)
    stationary_distribution = stationary_distribution[:, np.newaxis]
    q_function = compute_qfunction(agent, policy)
    value_function = compute_value_function(agent, policy)
    advantage = q_function - value_function[:, None]
    gradient = (1 / (1 - gamma)) * stationary_distribution * policy * advantage
    return gradient

def compute_objective(logits):
    policy = compute_policy(logits)
    objective = 0.0
    for agent in range(N):
        statinnary_distrubtion_agent = compute_stationnary_distribution(agent, policy)
        reward_mrp = compute_mrp_reward(agent, policy)
        objective += np.dot(statinnary_distrubtion_agent,reward_mrp)
    return(objective/N)

def compute_objective_agent_i(logits,agent):
    policy = compute_policy(logits)
    statinnary_distrubtion_agent = compute_stationnary_distribution(agent, policy)
    reward_mrp = compute_mrp_reward(agent, policy)
    objective = np.dot(statinnary_distrubtion_agent,reward_mrp)
    return objective

In [5]:
# execute federated policy gradient
nb_simulations = 10
logits = [np.random.randint(0, 10, size=(S, A)) for i in range(nb_simulations)]
for logit_value in logits:
    logit = deepcopy(logit_value)
    for t in range(number_rounds):
        logit_client_list = [deepcopy(logit) for i in range(N)]
        for agent in range(N):
            logit_client = deepcopy(logit_client_list[agent])
            for lstep in range(Local_steps):
                logit_client = logit_client + step_size * compute_gradient(agent, logit_client)
            logit_client_list[agent] = logit_client
        logit_avg = np.zeros((S, A))
        for i in range(N):
            logit_avg += logit_client_list[agent]
        logit = logit_avg/N
    print(compute_objective(logit))
    print(compute_policy(logit))

0.7758204927854486
[[9.99976699e-01 1.00533140e-05 1.32476896e-05]
 [1.64007207e-06 9.99989762e-01 8.59748588e-06]
 [2.03377798e-05 8.69108086e-06 9.99970971e-01]]
0.7758219773520033
[[9.99983886e-01 1.11753113e-05 4.93915664e-06]
 [2.18266305e-06 9.99994262e-01 3.55550607e-06]
 [2.01922177e-05 7.88618967e-06 9.99971922e-01]]
0.7758211548204447
[[9.99975882e-01 7.48825900e-06 1.66296242e-05]
 [1.59198847e-06 9.99992685e-01 5.72323769e-06]
 [1.51079749e-05 9.29436337e-06 9.99975598e-01]]
0.7758213042761619
[[9.99977758e-01 8.74006426e-06 1.35020928e-05]
 [1.66358725e-06 9.99992756e-01 5.58003325e-06]
 [1.74488638e-05 8.29616978e-06 9.99974255e-01]]
0.7758212597239353
[[9.99977335e-01 7.60154149e-06 1.50636751e-05]
 [1.62966546e-06 9.99992666e-01 5.70416509e-06]
 [1.41522159e-05 9.90098600e-06 9.99975947e-01]]
0.7758213874651696
[[9.99977317e-01 6.49236027e-06 1.61905261e-05]
 [1.86967198e-06 9.99993241e-01 4.88930453e-06]
 [9.77679175e-06 1.11259974e-05 9.99979097e-01]]
0.77582141634367

In [6]:
# execute policy gradient without aggregation
nb_simulations = 10
logits = [np.random.randint(0, 10, size=(S, A)) for i in range(nb_simulations)]
Initialisation = 0
for logit_value in logits:
    Initialisation +=1
    logit_client_list = [deepcopy(logit_value) for i in range(N)]
    for t in range(50*number_rounds):
        for agent in range(N):
            logit_client = deepcopy(logit_client_list[agent])
            logit_client = logit_client + step_size * compute_gradient(agent, logit_client)
            logit_client_list[agent] = logit_client
    print("Initialisation",Initialisation )
    for agent in range(N):
        print("Objective of agent", agent," when solo training:", compute_objective_agent_i(logit_client_list[agent],agent))
    print("--------------------------------------------------------------")

Initialisation 1
Objective of agent 0  when solo training: 0.8346119814922899
Objective of agent 1  when solo training: 0.8533465000653994
Objective of agent 2  when solo training: 0.8980503591190803
Objective of agent 3  when solo training: 0.8136753939838457
Objective of agent 4  when solo training: 0.8188249927967213
--------------------------------------------------------------
Initialisation 2
Objective of agent 0  when solo training: 0.8346119839875845
Objective of agent 1  when solo training: 0.85334648182004
Objective of agent 2  when solo training: 0.8980503791399955
Objective of agent 3  when solo training: 0.8136754114213736
Objective of agent 4  when solo training: 0.8188312056229217
--------------------------------------------------------------
Initialisation 3
Objective of agent 0  when solo training: 0.8346119947152308
Objective of agent 1  when solo training: 0.8462299020850155
Objective of agent 2  when solo training: 0.8980503717907287
Objective of agent 3  when solo 

# Attempt to find counter-examples

In [3]:
#Params
np.random.seed(1)
N=2
S, A = 3, 2 # state space
gamma = 0.9 #discount factor 
init_dist = np.array([0,0,1]) #init_distribution
number_rounds = 50000
Local_steps = 1
step_size  = 0.001
tau = 0.5

In [4]:
#Generate environnements
env1 = FiniteStateFiniteActionMDP(S=3, A=2)
env2 = FiniteStateFiniteActionMDP(S=3, A=2)
P_1 = env1.get_P()
r = env1.get_r()
P_1[0,0,0] = 1.0
P_1[0,0,1] = 0.0
P_1[0,0,2] = 0.0
P_1[0,1,0] = 1.0
P_1[0,1,1] = 0.0
P_1[0,1,2] = 0.0

P_1[1,0,0] = 0.0
P_1[1,0,1] = 1.0
P_1[1,0,2] = 0.0
P_1[1,1,0] = 0.0
P_1[1,1,1] = 1.0
P_1[1,1,2] = 0.0


P_1[2,0,0] = 0.0
P_1[2,0,1] = 0.0
P_1[2,0,2] = 1.0
P_1[2,1,0] = 0.0
P_1[2,1,1] = 1.0
P_1[2,1,2] = 0.0


#Second MDP
P_2 = env2.get_P()

P_2[0,0,0] = 1.0
P_2[0,0,1] = 0.0
P_2[0,0,2] = 0.0
P_2[0,1,0] = 1.0
P_2[0,1,1] = 0.0
P_2[0,1,2] = 0.0

P_2[1,0,0] = 0.0
P_2[1,0,1] = 1.0
P_2[1,0,2] = 0.0
P_2[1,1,0] = 0.0
P_2[1,1,1] = 1.0
P_2[1,1,2] = 0.0


P_2[2,0,0] = 1.0
P_2[2,0,1] = 0.0
P_2[2,0,2] = 0.0
P_2[2,1,0] = 0.0
P_2[2,1,1] = 0.0
P_2[2,1,2] = 1.0

r[0,0] = 10
r[0,1] = 10
r[1,0] = 10
r[1,1] = 10
r[2,0] = 0.0
r[2,1] = 0.0

env1.P = P_1
env2.P = P_2
env1.R = r
env2.R = r
envs = [env1, env2]

In [36]:
#Useful functions

def compute_mrp_transition(agent, policy):
    transition_kernel =  envs[agent].get_P()
    mrp_transition = np.sum(policy[:, :, np.newaxis] * transition_kernel, axis=1)
    return mrp_transition

def compute_mrp_reward(agent, policy):
    reward = envs[agent].get_r()
    # Element-wise multiplication and sum along the actions axis
    mrp_reward = np.sum(policy * reward, axis=1)
    return mrp_reward

def compute_stationnary_distribution(agent, policy):
    mrp_transition = compute_mrp_transition(agent, policy)
    stationnary_distribution = (1- gamma) *init_dist.T @ np.linalg.inv(np.eye(S) -gamma *mrp_transition) 
    return stationnary_distribution

def compute_value_function(agent, policy):
    mrp_transition = compute_mrp_transition(agent, policy)
    mrp_reward = compute_mrp_reward(agent, policy)
    return np.linalg.inv(np.eye(S) -gamma *mrp_transition) @ mrp_reward

def compute_qfunction(agent, policy):
    reward = envs[agent].get_r()
    transitions = envs[agent].get_P()
    value_function = compute_value_function(agent, policy)
    expected_future_rewards = np.sum(transitions * value_function[np.newaxis, np.newaxis, :], axis=2)
    Q_function = reward + gamma * expected_future_rewards
    return Q_function

def compute_policy(logits):
    logits_max = np.max(logits, axis=1, keepdims=True)
    exp_logits = np.exp(logits - logits_max)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

def compute_gradient(agent, logits):
    policy = compute_policy(logits)
    stationary_distribution = compute_stationnary_distribution(agent, policy)
    stationary_distribution = stationary_distribution[:, np.newaxis]
    print(stationary_distribution)
    q_function = compute_qfunction(agent, policy)
    value_function = compute_value_function(agent, policy)
    advantage = q_function - value_function[:, None]
    gradient = (1 / (1 - gamma)) * stationary_distribution * policy * advantage
    return gradient

def compute_objective(logits):
    policy = compute_policy(logits)
    objective = 0.0
    for agent in range(N):
        statinnary_distrubtion_agent = compute_stationnary_distribution(agent, policy)
        reward_mrp = compute_mrp_reward(agent, policy)
        objective += np.dot(statinnary_distrubtion_agent,reward_mrp)
    return(objective/N)

In [50]:
# execute policy gradient
nb_simulations = 1
logits = [np.random.randint(0, 10, size=(S, A)) for i in range(nb_simulations)]
logits[0][2] = [0.5,0.5]
gradient = np.zeros((S, A))
for agent in range(N):
    gradient +=compute_gradient(agent, logits[0])
#print("norm of the gradient at the beggining of the training", np.linalg.norm(gradient))
for logit_value in logits:
    logit = deepcopy(logit_value)
    for t in range(4):
        logit_client_list = [deepcopy(logit) for i in range(N)]
        gradient = np.zeros((S, A))
        for agent in range(N):
            logit_client = deepcopy(logit_client_list[agent])
            for lstep in range(Local_steps):
                gradient_agent = compute_gradient(agent, logit_client)
                gradient += gradient_agent
                logit_client = logit_client + step_size * gradient_agent
            logit_client_list[agent] = logit_client
        #print("gradient",gradient)
        #print("norm of the gradient", np.linalg.norm(gradient))
        print(logit_client_list)
        logit_avg = np.zeros((S, A))
        for i in range(N):
            logit_avg += logit_client_list[agent]
        logit = logit_avg/N
        print("objective", compute_objective(logit))
        print("policy",compute_policy(logit))

[array([[ 6.00000000e+00,  2.00000000e+00],
       [ 9.00000000e+00,  5.00000000e+00],
       [-7.43801653e-03,  7.43801653e-03]]), array([[ 6.00000000e+00,  2.00000000e+00],
       [ 9.00000000e+00,  5.00000000e+00],
       [ 7.43801653e-03, -7.43801653e-03]])]
objective 8.181750845199659
policy [[0.98201379 0.01798621]
 [0.98201379 0.01798621]
 [0.50371894 0.49628106]]
[array([[ 6.00000000e+00,  2.00000000e+00],
       [ 9.00000000e+00,  5.00000000e+00],
       [-9.09452488e-05,  9.09452488e-05]]), array([[ 6.        ,  2.        ],
       [ 9.        ,  5.        ],
       [ 0.01478592, -0.01478592]])]
objective 8.181552088614652
policy [[0.98201379 0.01798621]
 [0.98201379 0.01798621]
 [0.50739242 0.49260758]]
[array([[ 6.00000000e+00,  2.00000000e+00],
       [ 9.00000000e+00,  5.00000000e+00],
       [ 7.16629761e-03, -7.16629761e-03]]), array([[ 6.        ,  2.        ],
       [ 9.        ,  5.        ],
       [ 0.02204561, -0.02204561]])]
objective 8.181226645037288
policy [[

In [44]:
nb_simulations = 1
logits = [np.random.randint(0, 10, size=(S, A)) for i in range(nb_simulations)]
logits[0][2] = [1,1]
policy = compute_policy(logits[0])
stat_dist = compute_stationnary_distribution(1, compute_policy(logits[0]))
q_function_1 = compute_qfunction(0, policy)
value_function_1 = compute_value_function(0, policy)
q_function_2 = compute_qfunction(1, policy)
value_function_2 = compute_value_function(1, policy)
advantage1 = q_function_1 - value_function_1[:, None]
advantage2 = q_function_2 - value_function_2[:, None]
print(advantage1)
print(advantage2)
print(compute_mrp_transition(1, compute_policy(logits[0])))
print(stat_dist)

[[ 0.00000000e+00  0.00000000e+00]
 [ 1.42108547e-14  1.42108547e-14]
 [-8.18181818e+00  8.18181818e+00]]
[[ 0.00000000e+00  0.00000000e+00]
 [ 1.42108547e-14  1.42108547e-14]
 [ 8.18181818e+00 -8.18181818e+00]]
[[1.  0.  0. ]
 [0.  1.  0. ]
 [0.5 0.  0.5]]
[0.81818182 0.         0.18181818]


In [ ]:
nb_simulations = 20
logits = [np.random.randint(0, 10, size=(S, A)) for i in range(nb_simulations)]
logit = deepcopy(logits[11])
for t in range(10*number_rounds):
    logit_client_list = [deepcopy(logit) for i in range(N)]
    for agent in range(N):
        logit_client = deepcopy(logit_client_list[agent])
        for lstep in range(Local_steps):
            logit_client = logit_client + step_size * compute_gradient(agent, logit_client)
        logit_client_list[agent] = logit_client
    logit_avg = np.zeros((S, A))
    for i in range(N):
        logit_avg += logit_client_list[agent]
    logit = logit_avg/N
print(compute_objective(logit))